In [1]:
# 1: Install necessary libraries in the Colab environment
# Run this cell once to install networkx and plotly
!pip install networkx plotly

# 2: Import all required libraries
import networkx as nx
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import VBox, HBox
import math
import random
from google.colab import output
from collections import defaultdict

# IMPORTANT: Enable third-party widgets
# Necessary to display the interactive controls in Colab
output.enable_custom_widget_manager()

# 3: Classes and Functions

class DynamicAssociativeNetwork:
    """
    Represents a single cluster in a Dynamic Associative Network.
    """
    def __init__(self, name):
        self.name = name
        self.graph = nx.Graph()
        self.nodes = {}
        self.total_feedback = 0

    def add_node(self, node_name, attributes=None):
        if attributes is None:
            attributes = {}
        if node_name not in self.nodes:
            self.nodes[node_name] = {'state': 0, 'attributes': attributes, 'feedback': 0}
            self.graph.add_node(node_name, **attributes)

    def add_edge(self, node1, node2):
        self.graph.add_edge(node1, node2)


def create_jets_sharks_network():
    """
    Creates and populates the Jets and Sharks network.
    """
    network = DynamicAssociativeNetwork("Jets and Sharks")
    data = {
        "Art":    {"Gang": "Jets", "Age": "40s", "Edu": "JH",  "Marital": "Single",  "Occ": "Pusher"},
        "Al":     {"Gang": "Jets", "Age": "30s", "Edu": "JH",  "Marital": "Married", "Occ": "Burglar"},
        "Sam":    {"Gang": "Jets", "Age": "20s", "Edu": "HS",  "Marital": "Single",  "Occ": "Bookie"},
        "Clyde":  {"Gang": "Jets", "Age": "40s", "Edu": "JH",  "Marital": "Single",  "Occ": "Pusher"},
        "Mike":   {"Gang": "Jets", "Age": "30s", "Edu": "JH",  "Marital": "Single",  "Occ": "Bookie"},
        "Jim":    {"Gang": "Jets", "Age": "20s", "Edu": "JH",  "Marital": "Married", "Occ": "Burglar"},
        "Greg":   {"Gang": "Jets", "Age": "20s", "Edu": "HS",  "Marital": "Single",  "Occ": "Pusher"},
        "John":   {"Gang": "Jets", "Age": "20s", "Edu": "JH",  "Marital": "Married", "Occ": "Burglar"},
        "Doug":   {"Gang": "Jets", "Age": "30s", "Edu": "HS",  "Marital": "Single",  "Occ": "Bookie"},
        "Lance":  {"Gang": "Jets", "Age": "20s", "Edu": "JH",  "Marital": "Married", "Occ": "Burglar"},
        "George": {"Gang": "Jets", "Age": "20s", "Edu": "JH",  "Marital": "Married", "Occ": "Burglar"},
        "Pete":   {"Gang": "Jets", "Age": "20s", "Edu": "HS",  "Marital": "Single",  "Occ": "Bookie"},
        "Fred":   {"Gang": "Jets", "Age": "20s", "Edu": "HS",  "Marital": "Single",  "Occ": "Pusher"},
        "Gene":   {"Gang": "Jets", "Age": "20s", "Edu": "COL", "Marital": "Single",  "Occ": "Pusher"},
        "Ralph":  {"Gang": "Jets", "Age": "30s", "Edu": "JH",  "Marital": "Single",  "Occ": "Pusher"},
        "Bern":   {"Gang": "Sharks", "Age": "40s", "Edu": "HS",  "Marital": "Married", "Occ": "Pusher"},
        "Nick":   {"Gang": "Sharks", "Age": "30s", "Edu": "COL", "Marital": "Single",  "Occ": "Burglar"},
        "Ken":    {"Gang": "Sharks", "Age": "20s", "Edu": "HS",  "Marital": "Married", "Occ": "Bookie"},
        "Ike":    {"Gang": "Sharks", "Age": "30s", "Edu": "JH",  "Marital": "Single",  "Occ": "Bookie"},
        "Don":    {"Gang": "Sharks", "Age": "20s", "Edu": "COL", "Marital": "Married", "Occ": "Burglar"},
        "Ned":    {"Gang": "Sharks", "Age": "30s", "Edu": "COL", "Marital": "Married", "Occ": "Bookie"},
        "Karl":   {"Gang": "Sharks", "Age": "40s", "Edu": "HS",  "Marital": "Married", "Occ": "Pusher"},
        "Earl":   {"Gang": "Sharks", "Age": "30s", "Edu": "HS",  "Marital": "Married", "Occ": "Pusher"},
        "Phil":   {"Gang": "Sharks", "Age": "20s", "Edu": "HS",  "Marital": "Married", "Occ": "Burglar"},
        "Saul":   {"Gang": "Sharks", "Age": "20s", "Edu": "HS",  "Marital": "Single",  "Occ": "Bookie"},
        "Rick":   {"Gang": "Sharks", "Age": "30s", "Edu": "JH",  "Marital": "Single", "Occ": "Pusher"},
    }

    all_properties = defaultdict(set)
    for name, properties in data.items():
        network.add_node(name, attributes={'type': 'Person'})
        for prop_type, prop_value in properties.items():
            all_properties[prop_type].add(prop_value)

    for prop_type, prop_values in all_properties.items():
        for prop_value in prop_values:
            network.add_node(prop_value, attributes={'type': prop_type})

    for name, properties in data.items():
        for prop_value in properties.values():
            network.add_edge(name, prop_value)

    return network, data, all_properties


# 4: Create the network and perform pre-calculations for DAN model
network, data, all_properties = create_jets_sharks_network()
G = network.graph
person_nodes = [node for node, attrs in G.nodes(data=True) if attrs['type'] == 'Person']
property_nodes = {node: attrs for node, attrs in G.nodes(data=True) if attrs['type'] != 'Person'}

# DAN Pre-calculations
# 1. Calculate Diffusion for each property
property_diffusion = {}
for prop_name in property_nodes:
    count = len(list(G.neighbors(prop_name)))
    property_diffusion[prop_name] = 1 / count if count > 0 else 0

# 2. Calculate Peak Activation for each person
person_peak_activation = {}
for person_name in person_nodes:
    person_properties = data[person_name].values()
    excitation = sum(property_diffusion[prop] for prop in person_properties)
    # Infusion for peak activation is always 1/5 as it's based on the person's own 5 properties
    infusion = 1 / 5.0
    person_peak_activation[person_name] = excitation * infusion

# Visualization Setup (Coordinates and Initial Traces)
pos = {}

radius = 10; angle_increment = (2 * math.pi) / len(person_nodes); current_angle = 0
category_positions = {'Gang': {'x': 25, 'y': 0, 'z': 0}, 'Age': {'x': -25, 'y': 0, 'z': 0}, 'Edu': {'x': 0, 'y': 25, 'z': 0}, 'Marital': {'x': 0, 'y': -25, 'z': 0}, 'Occ': {'x': 0, 'y': 0, 'z': 25}}
for i, node_name in enumerate(person_nodes):
    pos[node_name] = (radius * math.cos(current_angle), radius * math.sin(current_angle), 0); current_angle += angle_increment
for node_name, attrs in G.nodes(data=True):
    if attrs['type'] != 'Person':
        cat_pos = category_positions[attrs['type']]; pos[node_name] = (cat_pos['x'] + (random.random() - 0.5) * 5, cat_pos['y'] + (random.random() - 0.5) * 5, cat_pos['z'] + (random.random() - 0.5) * 5)

edge_x, edge_y, edge_z = [], [], [];
for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]; x1, y1, z1 = pos[edge[1]]; edge_x.extend([x0, x1, None]); edge_y.extend([y0, y1, None]); edge_z.extend([z0, z1, None])
edge_trace = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z, line=dict(width=0.5, color='#888'), hoverinfo='none', mode='lines')

node_x, node_y, node_z, node_text = [], [], [], []; initial_node_colors = []; node_names_in_order = []
category_colors = {'Person': '#1f77b4', 'Gang': '#ff7f0e', 'Age': '#2ca02c', 'Edu': '#d62728', 'Marital': '#9467bd', 'Occ': '#8c564b'}
for node, attrs in G.nodes(data=True):
    x, y, z = pos[node]; node_x.append(x); node_y.append(y); node_z.append(z); node_text.append(f'{node}<br>Type: {attrs["type"]}'); initial_node_colors.append(category_colors.get(attrs['type'], '#7f7f7f')); node_names_in_order.append(node)
node_trace = go.Scatter3d(x=node_x, y=node_y, z=node_z, mode='markers', hoverinfo='text', text=node_text, marker=dict(color=initial_node_colors, size=10, line_width=1))

# 5: Create interactive FigureWidget and UI Components
fig = go.FigureWidget(data=[edge_trace, node_trace], layout=go.Layout(title='<br>Dynamic Associative Network: Select properties and run inference', showlegend=False, height=600, scene=dict(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''), yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''), zaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''))))

selection_widgets = {}
for prop_type, prop_values in sorted(all_properties.items()):
    selection_widgets[prop_type] = widgets.SelectMultiple(options=sorted(list(prop_values)), description=f'{prop_type}:', disabled=False, rows=len(prop_values))
run_button = widgets.Button(description="Run Inference", button_style='success')
reset_button = widgets.Button(description="Reset View", button_style='info')
results_output = widgets.Output()

# 6: Define core DAN logic and UI update functions
def run_inference(b):
    """This function runs the full DAN two-phase calculation."""
    results_output.clear_output()
    query_properties = []
    for w in selection_widgets.values():
        query_properties.extend(w.value)

    if not query_properties:
        reset_view(None)
        return

    # Part 1: Properties -> Objects
    object_weights = {}
    query_infusion = 1 / len(query_properties)
    for person_name in person_nodes:
        person_properties = data[person_name].values()
        # Calculate excitation based on the intersection of person's properties and the query
        excitation = sum(property_diffusion[prop] for prop in query_properties if prop in person_properties)
        activation = excitation * query_infusion
        # Normalize by peak activation to get the final weight
        object_weights[person_name] = activation / person_peak_activation[person_name] if person_peak_activation[person_name] > 0 else 0

    # Part 2: Objects -> Properties
    final_property_activations = defaultdict(float)
    object_infusion = 1 / 5.0 # Each person is defined by 5 properties
    for person_name, weight in object_weights.items():
        person_properties = data[person_name].values()
        for prop in person_properties:
            final_property_activations[prop] += weight * object_infusion

    # Update Visualization
    # Find top 5 predicted properties (excluding query properties)
    sorted_predictions = sorted(final_property_activations.items(), key=lambda item: item[1], reverse=True)
    winning_properties = [p[0] for p in sorted_predictions if p[0] not in query_properties][:5]

    # Update node colors
    updated_colors = ['#d3d3d3'] * len(node_names_in_order) # Grey out all
    for i, node_name in enumerate(node_names_in_order):
        if node_name in query_properties:
            updated_colors[i] = 'blue'  # Query
        elif node_name in winning_properties:
            updated_colors[i] = 'lime' # Prediction
        elif node_name in person_nodes and object_weights.get(node_name, 0) > 0.5:
             updated_colors[i] = '#ff69b4' # Activated Person

    with fig.batch_update():
        fig.data[1].marker.color = updated_colors

    # Display results
    with results_output:
        print("--- Inference Results ---")
        print("Query:", ", ".join(query_properties))
        print("\nTop Predicted Associations:")
        for prop, score in sorted_predictions:
            if prop not in query_properties:
                print(f"- {prop}: {score:.4f}")


def reset_view(b):
    """Resets the UI and graph to the initial state."""
    results_output.clear_output()
    for w in selection_widgets.values():
        w.value = []
    with fig.batch_update():
        fig.data[1].marker.color = initial_node_colors

# Link functions to buttons
run_button.on_click(run_inference)
reset_button.on_click(reset_view)

# 7: Display the UI
controls = HBox([HBox(list(selection_widgets.values())), VBox([run_button, reset_button], layout={'justify_content': 'center', 'margin': '0 0 0 20px'})])
app = VBox([controls, results_output, fig])
display(app)